# Метаклассы

Это классы, экземпляры которых являются классами

* type - Стандратный метакласс в Python
* Классы в Python - это объекты типа type

In [7]:
MyClass = type('MyClass', (object,), {})
print(MyClass)
print(type(MyClass))
obj = MyClass()
print(obj)

<class '__main__.MyClass'>
<class 'type'>


In [8]:
def my_class_init(instance, attr):
    instance.attr = attr

MyClass = type('MyClass', (object,), {'__init__': my_class_init})
obj = MyClass('Hello')
print(obj)
print(obj.attr)

Hello


* Можно создавать свои метаклассы. 
* Сгодится любой callable объект, принимающий 3 аргумента (имя, bases, аттрибуты) и возвращающий объект класса.
* Метакласс можно указать при объявлении класса
* Метаклассы запускаются перед созданием класса

In [14]:
def my_metaclass(name, parents, attributes):
    print(parents, attributes)
    return 'Hello'

class C(metaclass=my_metaclass):
    attr = 1
    
    def __init__(self):
        pass

() {'__module__': '__main__', '__qualname__': 'C', 'attr': 1, '__init__': <function C.__init__ at 0x7ff866b4d820>}


Оборачивает каждый метод класса декоратором logged (Логирование)

In [24]:
from functools import wraps
import time

def logged(time_format, name_prefix=""):
    def decorator(func):
        if hasattr(func, '_logged_decorator') and func._logged_decorator:
            return func

        @wraps(func)
        def decorated_func(*args, **kwargs):
            start_time = time.time()
            print("- Running '{}' on {} ".format(
                name_prefix + func.__name__,
                time.strftime(time_format)
            ))
            result = func(*args, **kwargs)
            end_time = time.time()
            print("- Finished '{}', execution time = {:0.3f}s ".format(
                name_prefix + func.__name__,
                end_time - start_time
            ))
            return result
        decorated_func._logged_decorator = True
        return decorated_func
    return decorator
# ....................................................................

def log_everything_metaclass(class_name, parents, attributes):
    print('Creating class', class_name)
    class_attributes = {}
    for attr_name, attr in attributes.items():
        class_attributes[attr_name] = attr
        if hasattr(attr, '__call__'):
            class_attributes[attr_name] = logged("%b %d %Y - %H:%M:%S",
                                            class_name + '.')(attr)
    return type(class_name, parents, class_attributes)


class C(metaclass=log_everything_metaclass):
    def __init__(self, x):
        self.x = x

    def print_x(self):
        print(self.x)
        

# Usage:
print('Starting object creation')
c = C('Test')
c.print_x()

Creating class C
Starting object creation
- Running 'C.__init__' on Oct 22 2021 - 23:13:31 
- Finished 'C.__init__', execution time = 0.000s 
- Running 'C.print_x' on Oct 22 2021 - 23:13:31 
Test
- Finished 'C.print_x', execution time = 0.000s 


###### С помощью метаклассов можно реализовать паттерн "Фабрика классов"

In [25]:
frametype_class_dict = {}

class ID3v2FrameClassFactory(type):
    def __new__(cls, class_name, parents, attributes):
        c = type(class_name, parents, attributes)
        if attributes['frame_identifier']:
            frametype_class_dict[attributes['frame_identifier']] = c
        return c
    
    @staticmethod
    def get_class_from_frame_identifier(frame_identifier):
        return frametype_class_dict.get(frame_identifier)
    

class ID3v2Frame(metaclass=ID3v2FrameClassFactory):
    frame_identifier = None


class ID3v2TitleFrame(ID3v2Frame, metaclass=ID3v2FrameClassFactory):
    frame_identifier = 'TIT2'


class ID3v2CommentFrame(ID3v2Frame, metaclass=ID3v2FrameClassFactory):
    frame_identifier = 'COMM'


title_class = ID3v2FrameClassFactory.get_class_from_frame_identifier('TIT2')
comment_class = ID3v2FrameClassFactory.get_class_from_frame_identifier('COMM')
print(title_class)
print(comment_class)

<class '__main__.ID3v2TitleFrame'>
<class '__main__.ID3v2CommentFrame'>


При вызове класса для создания нового объекта вызывается его функция \__call__. Она вызывает type.\__call__ для создания объекта.

In [86]:
class my_metaclass(type):
    def __new__(cls, class_name, parents, attributes):
        print('- my_metaclass.__new__ - Creating class instance of type', cls)
        return super().__new__(cls, class_name, parents, attributes)

    def __init__(self, class_name, parents, attributes):
        print('- my_metaclass.__init__ - Initializing the class instance', self)
        super().__init__(class_name, parents, attributes)

    def __call__(self, *args, **kwargs):
        print('- my_metaclass.__call__ - Creating object of type ', self)
        return super().__call__(*args, **kwargs)


def my_class_decorator(cls):
    print('- my_class_decorator - Chance to modify the class', cls)
    return cls


# @my_class_decorator
class C(metaclass=my_metaclass):

    def __new__(cls, *args, **kwargs):
        print('- C.__new__ - Creating object.')
        return super(C, cls).__new__(cls)

    def __init__(self, x=100):
        self.x = x
        print('- C.__init__ - Initializing object.')

c = C()
print('Object c =', c)

- my_metaclass.__new__ - Creating class instance of type <class '__main__.my_metaclass'>
- my_metaclass.__init__ - Initializing the class instance <class '__main__.C'>
- my_metaclass.__call__ - Creating object of type  <class '__main__.C'>
- C.__new__ - Creating object.
- C.__init__ - Initializing object.
Object c = <__main__.C object at 0x7ff88ea5ff70>


* Python читает определение класса и готовится передать три параметра в метакласс. Вот параметры: class_name, parents и attributes.
* В нашем случае метакласс представляет собой класс, поэтому его вызов похож на создание нового класса. Это значит, что первый my_metaclass.__new__ вызывается с четырьмя параметрами. Так создаётся объект, который и станет классом с именем C. У объекта вызывается __init__, а затем в переменную C записывается ссылка на объект.
* Затем Python смотрит на декораторы, которые можно применить к классу. В нашем случае есть только один декоратор. Python вызывает его, передаёт возвращённый из метакласса класс в качестве параметра. Класс заменяется объектом, который возвращается из декоратора.
* Тип класса будет таким же, как определено в метаклассе.
* Когда класс вызывается для создания нового объекта, Python ищет __call__ в метаклассе, так как тип класса — метакласс. В нашем случае my.metaclass.__call__ просто вызывает type.__call__, который создаёт объект из переданного класса.
* Затем type.__call__ создаёт объект. Для этого он ищет C.__new__ и запускает его.
* Возвращённый объект готов к использованию.
